In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load data

In [ ]:
data = pd.read_csv('../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')
data

# Data Analytic

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.countplot(data['DEATH_EVENT'])
plt.title('DEATH and ALIVE')
plt.show()

unbalance data

# Heat map

In [ ]:
plt.figure(figsize=(15,15))
cor = data.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

# Feature selection and Analyst some Feature

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import RFE

In [ ]:
print("all columns")
data.columns

In [ ]:
X=data.drop('DEATH_EVENT', 1)
y=data['DEATH_EVENT']

In [ ]:
cols = list(data.drop('DEATH_EVENT', 1).columns)
model = ExtraTreesClassifier()

# you can change feature selection model

rfe = RFE(model,5)             
# select 5 feature
X_rfe = rfe.fit_transform(X,y)  
model.fit(X_rfe,y)
print(model.feature_importances_)
temp = pd.Series(rfe.support_,index = cols)
selected_features_rfe = temp[temp==True].index



In [ ]:
print(selected_features_rfe)

AGE

In [ ]:
plt.figure(figsize=(6,8))
age = sns.boxplot(x="DEATH_EVENT", y="age",data=data)
age = sns.swarmplot(x="DEATH_EVENT", y="age", data=data, color="black",alpha=0.5)

ejection_fraction

In [ ]:
plt.figure(figsize=(6,8))
ejection = sns.boxplot(x="DEATH_EVENT", y="ejection_fraction",data=data)
ejection = sns.swarmplot(x="DEATH_EVENT", y="ejection_fraction", data=data, color="black",alpha=0.5)

serum_creatinine

In [ ]:
plt.figure(figsize=(6,8))
serum = sns.boxplot(x="DEATH_EVENT", y="serum_creatinine",data=data)
serum = sns.swarmplot(x="DEATH_EVENT", y="serum_creatinine", data=data, color="black",alpha=0.5)

serum_sodium

In [ ]:
plt.figure(figsize=(6,8))
serum = sns.boxplot(x="DEATH_EVENT", y="serum_sodium",data=data)
serum = sns.swarmplot(x="DEATH_EVENT", y="serum_sodium", data=data, color="black",alpha=0.5)

time

In [ ]:
plt.figure(figsize=(6,8))
serum = sns.boxplot(x="DEATH_EVENT", y="time",data=data)
serum = sns.swarmplot(x="DEATH_EVENT", y="time", data=data, color="black",alpha=0.5)

# Modeling

Train test split

In [ ]:
X=data[selected_features_rfe]
X

In [ ]:
y=data['DEATH_EVENT']
y

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=20,shuffle=True)

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

Find best parameter

In [ ]:
rfc=RandomForestClassifier(random_state=42)
param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)

In [ ]:
CV_rfc.best_params_

In [ ]:
rfc1=RandomForestClassifier(random_state=42, max_features='auto', n_estimators= 200, max_depth=4, criterion='entropy')
rfc1.fit(X_train, y_train)

In [ ]:
y_pred_rf=rfc1.predict(X_test)

In [ ]:
print('the accuracy of Test data is:',accuracy_score(y_test,y_pred_rf)*100,'%')

In [ ]:
print(confusion_matrix(y_test,y_pred_rf))
print(classification_report(y_test,y_pred_rf))

In [ ]:
cf_rf=metrics.confusion_matrix(y_test,y_pred_rf, labels=[1,0])
sns.heatmap(cf_rf,annot=True)

# LOGISTIC REGRESSION

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred_lr=lr.predict(X_test)
print('the accuracy of Test data is:',accuracy_score(y_test,y_pred_lr)*100,'%')

In [ ]:
print(confusion_matrix(y_test,y_pred_lr))
print(classification_report(y_test,y_pred_lr))

In [ ]:
cf_lr=metrics.confusion_matrix(y_test,y_pred_lr, labels=[1,0])
sns.heatmap(cf_lr,annot=True)

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

find best parameter

In [ ]:
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]
#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
#Create new KNN object
knn_2 = KNeighborsClassifier()
#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=10)
#Fit the model
best_model = clf.fit(X,y)
#Print The value of best Hyperparameters
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])

In [ ]:
knn= KNeighborsClassifier(n_neighbors=28)
knn.fit(X_train, y_train)
y_pred_knn=knn.predict(X_test)
print('the accuracy of Test data is:',accuracy_score(y_test,y_pred_knn)*100,'%')

In [ ]:
print(confusion_matrix(y_test,y_pred_knn))
print(classification_report(y_test,y_pred_knn))

In [ ]:
cf_lr=metrics.confusion_matrix(y_test,y_pred_knn, labels=[1,0])
sns.heatmap(cf_lr,annot=True)

# SVM

In [ ]:
from sklearn.svm import SVC


In [ ]:
# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf','linear']}  
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
grid.fit(X_train, y_train) 
# print best parameter after tuning 
print(grid.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_) 

In [ ]:
print(grid.best_estimator_) 

In [ ]:
y_pred_svm = grid.predict(X_test) 
print('the accuracy of Test data is:',accuracy_score(y_test,y_pred_svm)*100,'%')

In [ ]:
print(confusion_matrix(y_test,y_pred_svm))
print(classification_report(y_test,y_pred_svm))

In [ ]:
cf_svm=metrics.confusion_matrix(y_test,y_pred_svm, labels=[1,0])
sns.heatmap(cf_lr,annot=True)